In [1]:
!pip install torch

In [2]:
import numpy as np
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import dataset
import matplotlib.pyplot as plt
import os

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
models_path = '/content/gdrive/My Drive/models/'
model_name = 'scratch-sentiment-analysis-1.pth'
model_path = os.path.join(models_path,model_name)
print(model_path)

/content/gdrive/My Drive/models/scratch-sentiment-analysis-1.pth


In [ ]:
class MultiHeadAttention(nn.Module):
  def __init__(self,d_k,d_model,n_heads):
    super().__init__()
    self.d_k = d_k
    self.n_heads = n_heads

    self.key = nn.Linear(d_model,d_k*n_heads)
    self.query = nn.Linear(d_model,d_k*n_heads)
    self.value = nn.Linear(d_model,d_k*n_heads)

    self.fc = nn.Linear(d_k*n_heads,d_model)

  def forward(self,q,k,v,mask=None):
    q = self.query(q)
    k = self.key(k)
    v = self.value(v)

    N = q.shape[0]
    T = q.shape[1]

    q = q.view(N,T, self.n_heads,self.d_k).transpose(1,2)
    v = v.view(N,T, self.n_heads,self.d_k).transpose(1,2)
    k = k.view(N,T, self.n_heads,self.d_k).transpose(1,2)

    attn_scores = q @ k.transpose(-2,-1) / math.sqrt(self.d_k)

    if mask is not None:
      attn_scores = attn_scores.masked_fill(
          mask[:,None,None,:] ==0, float('-inf'))

    attn_weights = F.softmax(attn_scores,dim=-1)

    A = attn_weights @ v

    A = A.transpose(1,2)

    A = A.contiguous().view(N,T,self.d_k*self.n_heads)

    return self.fc(A)



In [ ]:
class TransformerBlock(nn.Module):
  def __init__(self,d_k,d_model,n_heads,dropout_prob=0.1):
    super().__init__()

    self.ln1 = nn.LayerNorm(d_model)
    self.ln2 = nn.LayerNorm(d_model)

    self.mha = MultiHeadAttention(d_k,d_model,n_heads)

    self.ann = nn.Sequential(
        nn.Linear(d_model,d_model*4),
        nn.GELU(),
        nn.Linear(d_model*4,d_model),
        nn.Dropout(dropout_prob)
    )

    self.dropout = nn.Dropout(p=dropout_prob)


  def forward(self,x,mask=None):
    x = self.ln1(x + self.mha(x,x,x,mask))
    x = self.ln2(x + self.ann(x))
    x = self.dropout(x)
    return x


In [ ]:
class PositionalEncoding(nn.Module):
  def __init__(self,d_model,max_len=2048,dropout_prob=0.1):
    super().__init__()
    self.dropout = nn.Dropout(p=dropout_prob)

    position = torch.arange(max_len).unsqueeze(1)
    exp_term = torch.arange(0,d_model,2)
    div_term = torch.exp(exp_term*(-math.log(10000.0)/d_model))
    pe = torch.zeros(1,max_len,d_model)
    pe[0,:,0::2] = torch.sin(position * div_term)
    pe[0,:,1::2] = torch.cos(position * div_term)
    self.register_buffer('pe',pe)

  def forward(self,x):
    x = x + self.pe[:,:x.size(1),:]
    return self.dropout(x)


In [ ]:
class Encoder(nn.Module):
  def __init__(self,vocab_size,max_len,d_k,d_model,n_heads,n_layers,n_classes,dropout_prob):
    super().__init__()

    self.embedding = nn.Embedding(vocab_size,d_model)

    self.pos_encoding = PositionalEncoding(d_model,max_len,dropout_prob)

    transformer_blocks = [
        TransformerBlock(d_k,d_model,n_heads,dropout_prob) for _ in range(n_layers)
    ]

    self.transformer_blocks = nn.Sequential(*transformer_blocks)
    self.ln = nn.LayerNorm(d_model)
    self.fc = nn.Linear(d_model,n_classes)


  def forward(self,x,mask=None):
    x = self.embedding(x)
    x = self.pos_encoding(x)
    for block in self.transformer_blocks:
        x = block(x,mask)

    #optional - this is for text classification (one output)
    x = x[:,0,:]

    x = self.ln(x)
    x = self.fc(x)
    return x


In [ ]:
model = Encoder(20_000,1024,16,64,4,2,5,0.1)

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
device

device(type='cuda', index=0)

In [ ]:
model.to(device)

Encoder(
  (embedding): Embedding(20000, 64)
  (pos_encoding): PositionalEncoding(
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (transformer_blocks): Sequential(
    (0): TransformerBlock(
      (ln1): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
      (ln2): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
      (mha): MultiHeadAttention(
        (key): Linear(in_features=64, out_features=64, bias=True)
        (query): Linear(in_features=64, out_features=64, bias=True)
        (value): Linear(in_features=64, out_features=64, bias=True)
        (fc): Linear(in_features=64, out_features=64, bias=True)
      )
      (ann): Sequential(
        (0): Linear(in_features=64, out_features=256, bias=True)
        (1): GELU(approximate='none')
        (2): Linear(in_features=256, out_features=64, bias=True)
        (3): Dropout(p=0.1, inplace=False)
      )
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (1): TransformerBlock(
      (ln1): LayerNorm((64,), eps=1e-05, 

In [ ]:
x = np.random.randint(0,20_000,size=(8,512))
x_t = torch.tensor(x).to(device)

In [ ]:
mask = np.ones((8,512))
mask[:,256:] = 0
mask_t = torch.tensor(mask).to(device)

In [ ]:
y= model(x_t,mask_t)

In [ ]:
y.shape

torch.Size([8, 5])

In [ ]:
!pip install transformers datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 42.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 492.4/492.4 kB 42.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 28.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 106.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 81.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 27.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.8 MB/s eta 0:00:00


In [ ]:
from transformers import AutoTokenizer, DataCollatorWithPadding

In [ ]:
checkpoint = 'distilbert-base-cased'
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [ ]:
from datasets import load_dataset

In [ ]:
raw_datasets = load_dataset('glue','sst2')
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Generating train split:   0%|          | 0/67349 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/872 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1821 [00:00<?, ? examples/s]

In [ ]:
def tokenize_fn(batch):
  return tokenizer(batch['sentence'],truncation=True)

In [ ]:
tokenized_datasets = raw_datasets.map(tokenize_fn,batched=True)

Map:   0%|          | 0/67349 [00:00<?, ? examples/s]

Map:   0%|          | 0/872 [00:00<?, ? examples/s]

Map:   0%|          | 0/1821 [00:00<?, ? examples/s]

In [ ]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'idx', 'input_ids', 'attention_mask'],
        num_rows: 67349
    })
    validation: Dataset({
        features: ['sentence', 'label', 'idx', 'input_ids', 'attention_mask'],
        num_rows: 872
    })
    test: Dataset({
        features: ['sentence', 'label', 'idx', 'input_ids', 'attention_mask'],
        num_rows: 1821
    })
})

In [ ]:
tokenized_datasets = tokenized_datasets.remove_columns(["sentence","idx"])
tokenized_datasets = tokenized_datasets.rename_column("label","labels")

In [ ]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 67349
    })
    validation: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 872
    })
    test: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 1821
    })
})

In [ ]:
from torch.utils.data import DataLoader

In [ ]:
train_loader = DataLoader(
    tokenized_datasets['train'],
    shuffle=True,
    batch_size=32,
    collate_fn=data_collator
)

valid_loader = DataLoader(
    tokenized_datasets['validation'],
    batch_size=32,
    collate_fn=data_collator
)

In [ ]:
model = Encoder(
    vocab_size=tokenizer.vocab_size,
    max_len=tokenizer.max_model_input_sizes[checkpoint],
    d_k = 16,
    d_model=64,
    n_heads=4,
    n_layers=2,
    n_classes=2,
    dropout_prob=0.1
)

In [ ]:
model.to(device)

Encoder(
  (embedding): Embedding(28996, 64)
  (pos_encoding): PositionalEncoding(
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (transformer_blocks): Sequential(
    (0): TransformerBlock(
      (ln1): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
      (ln2): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
      (mha): MultiHeadAttention(
        (key): Linear(in_features=64, out_features=64, bias=True)
        (query): Linear(in_features=64, out_features=64, bias=True)
        (value): Linear(in_features=64, out_features=64, bias=True)
        (fc): Linear(in_features=64, out_features=64, bias=True)
      )
      (ann): Sequential(
        (0): Linear(in_features=64, out_features=256, bias=True)
        (1): GELU(approximate='none')
        (2): Linear(in_features=256, out_features=64, bias=True)
        (3): Dropout(p=0.1, inplace=False)
      )
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (1): TransformerBlock(
      (ln1): LayerNorm((64,), eps=1e-05, 

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

In [ ]:
from datetime import datetime

In [ ]:
def train(model,criterion,optimizer,train_loader,valid_loader,epochs):
  train_losses = np.zeros(epochs)
  test_losses = np.zeros(epochs)

  for it in range(epochs):
    model.train()
    t0 = datetime.now()
    train_loss = 0
    n_train = 0
    for batch in train_loader:
      batch = {k: v.to(device) for k,v in batch.items()}
      optimizer.zero_grad()
      outputs = model(batch['input_ids'],batch['attention_mask'])
      loss = criterion(outputs,batch['labels'])
      loss.backward()
      optimizer.step()
      train_loss += loss.item()*batch['input_ids'].size(0)
      n_train += batch['input_ids'].size(0)
    train_loss = train_loss / n_train
    model.eval()
    test_loss = 0
    n_test = 0
    for batch in valid_loader:
      batch = {k: v.to(device) for k,v in batch.items()}
      outputs = model(batch['input_ids'],batch['attention_mask'])
      loss = criterion(outputs,batch['labels'])
      test_loss += loss.item()*batch['input_ids'].size(0)
      n_test += batch['input_ids'].size(0)
    test_loss = test_loss / n_test

    train_losses[it] = train_loss
    test_losses[it] = test_loss

    dt = datetime.now() - t0
    print(f'Epoch {it+1}/{epochs}, Train Loss: {train_loss:.4f} , Test Loss: {test_loss:.4f}, Duration: {dt}')
  return train_losses, test_losses

In [ ]:
train_losses,test_losses = train(model,criterion,optimizer,train_loader,valid_loader,epochs=4)
torch.save(model, model_path)

Epoch 1/4, Train Loss: 0.2274 , Test Loss: 0.1535, Duration: 0:00:31.838832
Epoch 2/4, Train Loss: 0.2042 , Test Loss: 0.1335, Duration: 0:00:28.081485
Epoch 3/4, Train Loss: 0.1870 , Test Loss: 0.1215, Duration: 0:00:31.240122
Epoch 4/4, Train Loss: 0.1732 , Test Loss: 0.1033, Duration: 0:00:26.712080


In [ ]:
torch.save(model, model_path)

In [ ]:
test_losses

array([0.15348581, 0.13352707, 0.12148538, 0.10333459])

In [ ]:
model.eval()
n_correct = 0
n_total = 0
for batch in train_loader:
  batch = {k: v.to(device) for k,v in batch.items()}
  outputs = model(batch['input_ids'],batch['attention_mask'])
  _,predictions = torch.max(outputs,1)
  n_correct += (predictions == batch['labels']).sum().item()
  n_total += batch['labels'].shape[0]

train_acc = n_correct / n_total

n_correct = 0
n_total = 0
for batch in valid_loader:
  batch = {k: v.to(device) for k,v in batch.items()}
  outputs = model(batch['input_ids'],batch['attention_mask'])
  _,predictions = torch.max(outputs,1)
  n_correct += (predictions == batch['labels']).sum().item()
  n_total += batch['labels'].shape[0]

test_acc = n_correct / n_total

print(f'train acc: {train_acc:.4f}, test acc: {test_acc:.4f}')

train acc: 0.9628, test acc: 0.7947
